In [1]:
###### 前処理本6章 nysol.mcmdによる処理
# 6.1 本章で共通して利用するライブラリの読み込み。
import os
import nysol.mcmd as nm
from glob import glob

os.makedirs('out_m',exist_ok=True)


In [2]:
# コード 6.2,6.3 収益率と超過収益率の計算方法
indexCSV='./in/index.csv'
closeCSV='./in/close.csv'
oCSV='./out_m/6-2.csv'

# ファイルの先頭をHTML形式で表示
display(nm.p(indexCSV))
display(nm.p(closeCSV))

# インデックスの収益率を求める:ireturn
idx=None
idx<<=nm.mcut(f='date,close',i=indexCSV)
# mslideは下の行の値をずらして持ってくるので並びを逆にして(date%r)ずらすことで前日の終値を当日の行にセットできる
idx<<=nm.mslide(s='date%r',f='close:close2')
idx<<=nm.mcal(c='${close}/${close2}-1',a='ireturn')

# 個別銘柄別に収益率を求める:return
f=None
f<<=nm.mcut(f='ticker,date,close',i=closeCSV)
f<<=nm.mslide(k='ticker',s='date%r',f='close:close2')
f<<=nm.mcal(c='${close}/${close2}-1',a='return')

# ireturnを結合して釣果収益率を求める
f<<=nm.mjoin(k='date',m=idx,f='ireturn')
f<<=nm.mcal(c='${return}-${ireturn}',a='areturn')
f<<=nm.mcut(f='ticker,date,return,ireturn,areturn',o=oCSV)

# 実行
f.run(msg='on')
display(nm.p(oCSV))


,date,close
0,20070417,400
1,20070418,450
2,20070419,460
3,20070420,430


,ticker,date,close
0,0001,20070417,100
1,0001,20070418,120
2,0001,20070419,120
3,0001,20070420,130
4,0002,20070417,5200
5,0002,20070418,5210


#END# kgcut f=ticker,date,close i=./in/close.csv; IN=8 OUT=8; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgcut f=date,close i=./in/index.csv; IN=4 OUT=4; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgslide f=close:close2 k=ticker s=date%r; IN=8 OUT=6; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgcal a=return c=${close}/${close2}-1; IN=6 OUT=6; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgslide f=close:close2 s=date%r; IN=4 OUT=3; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgcal a=ireturn c=${close}/${close2}-1; IN=3 OUT=3; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgjoin f=ireturn k=date; IN=6 OUT=6; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgcal a=areturn c=${return}-${ireturn}; IN=6 OUT=6; 2021/05/18 21:33:35; 2021/05/18 21:33:35
#END# kgcut f=ticker,date,return,ireturn,areturn o=./out_m/6-2.csv; IN=6 OUT=6; 2021/05/18 21:33:35; 2021/05/18 21:33:35


,ticker,date,return,ireturn,areturn
0,0001,20070418,0.2,0.125,0.075
1,0002,20070418,0.001923076923,0.125,-0.1230769231
2,0001,20070419,0,0.02222222222,-0.02222222222
3,0002,20070419,-0.00575815739,0.02222222222,-0.02798037961
4,0001,20070420,0.08333333333,-0.0652173913,0.1485507246
5,0002,20070420,-0.005791505792,-0.0652173913,0.05942588551


In [3]:
# コード 6.4 リストと辞書を使った分配金調整
# 関数として実装している。
def adjustDivi(priceCSV,diviCSV):
    # 配当金の累積処理
    divi=None
    divi<<=nm.maccum(s='date',f='dividend:div_fill',i=diviCSV)
    
    f=None
    # 株価データにdateをキーにしてaccumを結合しdividendという名前に変更
    # n=Trueとすることで、左結合(diviにdateがなければnull出力)となる
    f<<=nm.mjoin(k='date',m=divi,f='div_fill',n=True,i=priceCSV)
    # mnulltoでp=Trueとすれば、null値を前行の値に置き換えてくれる
    f<<=nm.mnullto(f='div_fill',p=True)
    # 先頭行のnullを0に置換する
    f<<=nm.mnullto(f='div_fill',v=0)
    # 価額調整の実行(価額+配当金)
    f<<=nm.mcal(c='${price}+${div_fill}',a='price_adj',o='out_m/adjustDivi.csv')
    # run()の戻り値は出力ファイル名となる
    ofile=f.run(msg='on')
    
    return ofile # ファイル名を返す

priceCSV='./in/price.csv'
diviCSV ='./in/dividend.csv'
price_adjCSV=adjustDivi(priceCSV,diviCSV)
print(price_adjCSV)
nm.p(price_adjCSV,head=10)


out_m/adjustDivi.csv


#END# kgaccum f=dividend:div_fill i=./in/dividend.csv s=date; IN=3 OUT=3; 2021/05/18 21:33:42; 2021/05/18 21:33:42
#END# kgnullto -p f=div_fill; IN=7 OUT=7; 2021/05/18 21:33:42; 2021/05/18 21:33:42
#END# kgjoin -n f=div_fill i=./in/price.csv k=date; IN=7 OUT=7; 2021/05/18 21:33:42; 2021/05/18 21:33:42
#END# kgcal a=price_adj c=${price}+${div_fill} o=out_m/adjustDivi.csv; IN=7 OUT=7; 2021/05/18 21:33:42; 2021/05/18 21:33:42
#END# kgnullto f=div_fill v=0; IN=7 OUT=7; 2021/05/18 21:33:42; 2021/05/18 21:33:42


,date,price,div_fill,price_adj
0,20070417,100,0,100
1,20070418,120,2,122
2,20070419,110,2,112
3,20070420,130,7,137
4,20070423,120,7,127
5,20070424,140,8,148
6,20070425,138,8,146


In [5]:
# (本書のものと同じ)
# コード 2.1、コード 2.2の再掲(インターネットに接続できなければ実行できない)

# 以下のURLでにアクセスすれば、３要因データの様々なバリエーション、
# weeklyやmonthly、そして他の国用のデータも揃っている。
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
import urllib
import zipfile

url='http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Japan_3_Factors_Daily_CSV.zip'
urllib.request.urlretrieve(url,'./out_m/3factor.zip')

with zipfile.ZipFile('./out_m/3factor.zip') as zf:
    # zipファイルの中にアーカイブされているフィアル一覧を表示する
    print(zf.namelist())
    # zipファイルから取り出すファイル名と、取り出し先にパス名を指定する。
    zf.extract('Japan_3_Factors_Daily.csv', './out_m')


['Japan_3_Factors_Daily.csv']


In [6]:
# コード 6.5 3要因モデルデータのクリーニング処理
fact3CSV='./out_m/3factors.csv'

f=None
# nm.cmdを使うことで標準入出力を入出力可能な多くのコマンドを利用できる
# awkは柔軟な表志向データのテキスト処理コマンドでNRは行番号を意味する
f<<=nm.cmd('awk "NR>7" ./Japan_3_Factors_Daily.csv')
# trは単一文字の置換/削除コマンドで、空白文字を削除している
f<<=nm.cmd('tr -d " "')
# dos改行になっているので、行末の'\r'を削除しておく
f<<=nm.cmd('tr -d "\r"')
# 入力データに項目名ヘッダーがない時は、nfni=Trueとすれば、
# 項目を左から0,1,2,...と指定できる。
# そして、出力は、項目名ヘッダーを出力するが、その名称を0:dateのようにして指定している
f<<=nm.mcut(f='0:date,1:Mkt-RF,2:SMB,3:HML,4:RF',nfni=True,o=fact3CSV)
f.run()

# 先頭行の確認
nm.p(fact3CSV)



,date,Mkt-RF,SMB,HML,RF
0,19900702,0.85,0.38,-0.06,0.03
1,19900703,0.07,0.72,0.30,0.03
2,19900704,1.45,0.52,0.26,0.03
3,19900705,-0.64,0.48,0.38,0.03
4,19900706,-0.02,0.40,0.31,0.03
5,19900709,-0.03,0.62,0.44,0.03


In [7]:
# (本書の内容と同じ)
# コード 6.6 ファイル名リストを作成するスクリプト
# ファイルリストの作成

def getTickers(iPath):
    tickers={}
    # globでワイルドカードにマッチする全ての「分配金」ファイル名を取得する。
    # *は任意の文字列にマッチすることを意味する。
    # for文にて、ファイル名を一つづつdivFile変数にセットして、インデントブロックの処理を回していく。
    for diviFile in glob(iPath+'*/*/*_divi.csv'):
        # pathの区切り文字である'/'でファイル名を分割し、全要素をnames変数にリストで格納する。
        # windowsとunix系OSで区切り文字が異なるため、共通のコードを書くためにos.sepを用いている。
        names=diviFile.split(os.sep) # ['.', 'in', '7', 'dividend', 'large', 'value', '0007_divi.csv']
        # tickerシンボル、value/gloth、small/largeの各属性値を変数にセットする。
        # リストのマイナスの数字は、後ろから数えての要素を取得している。
        ticker=names[-1].replace('_divi.csv','')
        gv    =names[-2]
        size  =names[-3]
        # 価額ファイル名は、分配金ファイル名の末尾から'_divi'を削除したものなので、ここでセットしておく。
        priceFile=diviFile.replace('_divi','')
        # tickerシンボルをキーにして属性を辞書にセットする。
        tickers[ticker]=[priceFile,diviFile,size,gv]
    # for文を抜けて、全ての銘柄がセットされたtickers辞書を返す。
    return tickers

fileList=getTickers('./in/dividend/')
fileList


{'0007': ['./in/dividend/large/value/0007.csv',
  './in/dividend/large/value/0007_divi.csv',
  'large',
  'value'],
 '0008': ['./in/dividend/large/value/0008.csv',
  './in/dividend/large/value/0008_divi.csv',
  'large',
  'value'],
 '0006': ['./in/dividend/large/growth/0006.csv',
  './in/dividend/large/growth/0006_divi.csv',
  'large',
  'growth'],
 '0005': ['./in/dividend/large/growth/0005.csv',
  './in/dividend/large/growth/0005_divi.csv',
  'large',
  'growth'],
 '0003': ['./in/dividend/small/value/0003.csv',
  './in/dividend/small/value/0003_divi.csv',
  'small',
  'value'],
 '0004': ['./in/dividend/small/value/0004.csv',
  './in/dividend/small/value/0004_divi.csv',
  'small',
  'value'],
 '0001': ['./in/dividend/small/growth/0001.csv',
  './in/dividend/small/growth/0001_divi.csv',
  'small',
  'growth'],
 '0002': ['./in/dividend/small/growth/0002.csv',
  './in/dividend/small/growth/0002_divi.csv',
  'small',
  'growth']}

In [8]:
# コード 6.7 全ファイルの価額調整をし、3要因モデルを結合したデータセットを作成

# 作成されたデータセットを保存するフォルダの作成。
os.makedirs('./out_m/adjust/',exist_ok=True)

# 直前のセルにて作成したファイル名一覧fileListで回す。
# keyは銘柄コード、valは価額ファイル名や分配金ファイル名などその属性リスト。
for key,val in fileList.items():
    # わかりやすさのため、リストの要素を変数にセットする。
    priceCSV=val[0]
    diviCSV=val[1]
    size=val[2]
    gvType=val[3]
    oCSV='./out_m/adjust/'+key+'.csv'
    
    # 前に作成した分配金調整の関数を呼び出す。
    fname=adjustDivi(priceCSV,diviCSV)
    f=None
    # 収益率の計算
    f<<=nm.mslide(s='date%r',f='price_adj:price_adj2',i=fname)
    f<<=nm.mcal(c='${price_adj}/${price_adj2}-1',a='return')
    # ３要因データの結合
    f<<=nm.mjoin(k='date',m=fact3CSV)
    # 各種属性値のセット
    f<<=nm.msetstr(v=[key,size,gvType],a='ticker,size,gvType')
    # 列の並びを整理する。
    f<<=nm.mcut(f='date,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML' ,o=oCSV)
    
    # 実行して中身確認
    f.run()
    display(nm.p(oCSV))
 

#END# kgaccum f=dividend:div_fill i=./in/dividend/large/value/0007_divi.csv s=date; IN=18 OUT=18; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgjoin -n f=div_fill i=./in/dividend/large/value/0007.csv k=date; IN=4832 OUT=4832; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgnullto -p f=div_fill; IN=4832 OUT=4832; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgnullto f=div_fill v=0; IN=4832 OUT=4832; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgcal a=price_adj c=${price}+${div_fill} o=out_m/adjustDivi.csv; IN=4832 OUT=4832; 2021/05/18 21:47:55; 2021/05/18 21:47:55


,date,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
0,19960401,0007,large,value,9675,0,9675,-0.0325,0.48,0.21,0.12
1,19960402,0007,large,value,9675,0,9675,0,0.00,0.61,0.21
2,19960403,0007,large,value,9753,0,9753,0.008062015504,0.45,0.55,-0.04
3,19960404,0007,large,value,9661,0,9661,-0.009432994976,-0.33,0.61,-0.18
4,19960405,0007,large,value,10000,0,10000,0.03508953524,0.42,-0.05,-0.13
5,19960408,0007,large,value,10404,0,10404,0.0404,-0.89,0.19,-0.11


#END# kgaccum f=dividend:div_fill i=./in/dividend/large/value/0008_divi.csv s=date; IN=22 OUT=22; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgjoin -n f=div_fill i=./in/dividend/large/value/0008.csv k=date; IN=6025 OUT=6025; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgnullto -p f=div_fill; IN=6025 OUT=6025; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgnullto f=div_fill v=0; IN=6025 OUT=6025; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgcal a=price_adj c=${price}+${div_fill} o=out_m/adjustDivi.csv; IN=6025 OUT=6025; 2021/05/18 21:47:55; 2021/05/18 21:47:55


,date,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
0,19910604,0008,large,value,9757,0,9757,-0.0243,-0.51,0.48,0.30
1,19910605,0008,large,value,9925,0,9925,0.0172184073,-1.07,0.41,0.10
2,19910606,0008,large,value,10000,0,10000,0.007556675063,-0.62,-0.22,0.31
3,19910607,0008,large,value,10278,0,10278,0.0278,-0.78,0.27,0.06
4,19910610,0008,large,value,10454,0,10454,0.01712395408,-2.42,-0.09,0.50
5,19910611,0008,large,value,10739,0,10739,0.02726229195,0.64,-0.18,0.01


#END# kgaccum f=dividend:div_fill i=./in/dividend/large/growth/0006_divi.csv s=date; IN=28 OUT=28; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgjoin -n f=div_fill i=./in/dividend/large/growth/0006.csv k=date; IN=3439 OUT=3439; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgnullto -p f=div_fill; IN=3439 OUT=3439; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgnullto f=div_fill v=0; IN=3439 OUT=3439; 2021/05/18 21:47:55; 2021/05/18 21:47:55
#END# kgcal a=price_adj c=${price}+${div_fill} o=out_m/adjustDivi.csv; IN=3439 OUT=3439; 2021/05/18 21:47:55; 2021/05/18 21:47:55


...

,date,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
0,20011031,0004,small,value,9927,0,9927,-0.0073,-1.05,0.54,0.79
1,20011101,0004,small,value,10590,0,10590,0.06678754911,0.17,-0.39,-0.97
2,20011102,0004,small,value,10163,0,10163,-0.0403210576,-0.09,0.11,-0.55
3,20011105,0004,small,value,9971,0,9971,-0.01889205943,0.08,-0.06,0.77
4,20011106,0004,small,value,9912,0,9912,-0.005917159763,1.22,-0.35,-1.20
5,20011107,0004,small,value,10045,0,10045,0.0134180791,-2.17,1.50,1.14


#END# kgaccum f=dividend:div_fill i=./in/dividend/small/growth/0001_divi.csv s=date; IN=13 OUT=13; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgjoin -n f=div_fill i=./in/dividend/small/growth/0001.csv k=date; IN=1571 OUT=1571; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgnullto -p f=div_fill; IN=1571 OUT=1571; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgnullto f=div_fill v=0; IN=1571 OUT=1571; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgcal a=price_adj c=${price}+${div_fill} o=out_m/adjustDivi.csv; IN=1571 OUT=1571; 2021/05/18 21:47:56; 2021/05/18 21:47:56


,date,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
0,20090703,0001,small,growth,9933,0,9933,-0.0067,-0.41,0.37,0.11
1,20090706,0001,small,growth,9888,0,9888,-0.004530353368,-0.08,0.76,-1.07
2,20090707,0001,small,growth,9944,0,9944,0.005663430421,0.24,0.27,-0.99
3,20090708,0001,small,growth,10373,0,10373,0.04314159292,-0.08,0.85,-1.51
4,20090709,0001,small,growth,10258,0,10258,-0.0110864745,-1.68,0.63,-0.93
5,20090710,0001,small,growth,10456,0,10456,0.01930200819,0.37,-0.18,0.34


#END# kgaccum f=dividend:div_fill i=./in/dividend/small/growth/0002_divi.csv s=date; IN=7 OUT=7; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgjoin -n f=div_fill i=./in/dividend/small/growth/0002.csv k=date; IN=1815 OUT=1815; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgnullto -p f=div_fill; IN=1815 OUT=1815; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgnullto f=div_fill v=0; IN=1815 OUT=1815; 2021/05/18 21:47:56; 2021/05/18 21:47:56
#END# kgcal a=price_adj c=${price}+${div_fill} o=out_m/adjustDivi.csv; IN=1815 OUT=1815; 2021/05/18 21:47:56; 2021/05/18 21:47:56


,date,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
0,20080703,0002,small,growth,9801,0,9801,-0.0199,-1.12,-0.41,-0.26
1,20080704,0002,small,growth,9366,0,9366,-0.0443832262,-0.06,0.05,0.26
2,20080707,0002,small,growth,9584,0,9584,0.02327567798,0.68,-0.30,-0.07
3,20080708,0002,small,growth,9783,0,9783,0.02076377295,-2.35,0.00,0.28
4,20080709,0002,small,growth,9783,0,9783,0,0.71,-0.13,0.04
5,20080710,0002,small,growth,10019,0,10019,0.02412347951,0.05,0.00,-0.20


In [9]:
# 章末問題 1) 銘柄別に収益率と超過収益率の平均と標準偏差を求めなさい

# コード6-2の結果はファイルに保存されているので、それを使う
iCSV='./out_m/6-2.csv'
# ticker,date%0,return,ireturn,areturn
# 0001,20070418,0.2,0.125,0.075
# 0002,20070418,0.001923076923,0.125,-0.1230769231
# 0001,20070419,0,0.02222222222,-0.02222222222

oCSV='./out_m/6-Q-1_mean.csv'
f=None
f<<=nm.mcut(f='ticker,return,areturn',i=iCSV)
f<<=nm.mstats(c='mean',k='ticker',f='return,areturn',o=oCSV)
f.run()
display(nm.p(oCSV))

oCSV='./out_m/6-Q-1_sd.csv'
f=None
f<<=nm.mcut(f='ticker,return,areturn',i=iCSV)
f<<=nm.mstats(c='usd',k='ticker',f='return,areturn',o=oCSV)
f.run()
display(nm.p(oCSV))

# msummaryで必要な統計量を指定すれば基本統計量を一気に出力できる。
oCSV='./out_m/6-Q-1_summary.csv'
stats='sum,mean,count,ucount,devsq,var,uvar,sd,usd,cv,min,qtile1,median,qtile3,max,range,qrange,mode,skew,uskew,kurt,ukurt'
f=None
f<<=nm.mcut(f='ticker,return,areturn',i=iCSV)
f<<=nm.msummary(c=stats,k='ticker',f='return,areturn',o=oCSV)
f.run()
display(nm.p(oCSV))


,ticker,return,areturn
0,0001,0.09444444444,0.06710950079
1,0002,-0.003208862086,-0.03054380573


,ticker,return,areturn
0,0001,0.1004618962,0.08565947014
1,0002,0.004444420831,0.09127840464


,ticker,fld,sum,mean,count,ucount,devsq,var,uvar,sd,usd,cv,min,qtile1,median,qtile3,max,range,qrange,mode,skew,uskew,kurt,ukurt
0,0001,return,0.2833333333,0.09444444444,3,3,0.02018518519,0.006728395062,0.01009259259,0.08202679478,0.1004618962,86.85190036,0,0.04166666666,0.08333333333,0.1416666667,0.2,0.2,0.1,,0.2007001872,0.49161305,-1.5,
1,0001,areturn,0.2013285024,0.06710950079,3,3,0.01467508965,0.00489169655,0.007337544825,0.06994066449,0.08565947014,104.2187226,-0.02222222222,0.02638888889,0.075,0.1117753623,0.1485507246,0.1707729468,0.08538647341,,-0.16778967,-0.4109990756,-1.5,
2,0002,return,-0.009626586259,-0.003208862086,3,3,3.950575305e-05,1.316858435e-05,1.975287653e-05,0.003628854413,0.004444420831,-113.0885129,-0.005791505792,-0.005774831591,-0.00575815739,-0.001917540234,0.001923076923,0.007714582715,0.003857291357,,0.7070619939,1.731941102,-1.5,
3,0002,areturn,-0.0916314172,-0.03054380573,3,3,0.01666349431,0.005554498102,0.008331747153,0.0745285053,0.09127840464,-244.0053016,-0.1230769231,-0.07552865136,-0.02798037961,0.01572275295,0.05942588551,0.1825028086,0.09125140431,,-0.05155217527,-0.1262765245,-1.5,


In [10]:
# 章末問題 2) 収益率の定義を「収益率=close_t/close_{t-1}」として計算する。

# コード 6.2,6.3 収益率と超過収益率の計算方法
indexCSV='./in/index.csv'
closeCSV='./in/close.csv'
oCSV='./out_m/6-Q-2.csv'

# 個別銘柄別に収益率を求める:return
f=None
f<<=nm.mcut(f='ticker,date,close',i=closeCSV)
# 最初の行をnullのまま出力するためにn=Trueを指定
f<<=nm.mslide(k='ticker',s='date%r',n=True,f='close:close2')
# ここの式がコード6.3とは異なる(-1していない)
f<<=nm.mcal(c='${close}/${close2}',a='return')

# ここから検証
# tickerが同じ値の行の先頭行にtopという項目名で1を出力
f<<=nm.mkeybreak(k='ticker',s='date')
# topがnullということは先頭行以外で、そのときは前の結果(#{})にreturnを掛ける
# topの時はcloseの値を出力する
f<<=nm.mcal(c='if(isnull(${top}),#{}*${return},${close})',a='close3',o=oCSV)
# closeとclose3の値が一致するはずである。

# 実行
f.run()
display(nm.p(oCSV))


,ticker,date,close,close2,return,top,bot,close3
0,0001,20070417,100,,,1,,100
1,0001,20070418,120,100,1.2,,,120
2,0001,20070419,120,120,1,,,120
3,0001,20070420,130,120,1.083333333,,1,130
4,0002,20070417,5200,,,1,,5200
5,0002,20070418,5210,5200,1.001923077,,,5210
